In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os, sys, gc
import pickle

import itertools
import seaborn as sns

from frontiers_colors import get_color
from frontiers_analysis import heaps, load_tissue, save_model, mazzolini

from tableanalyser import discretize_df_columns, plotvarmen, plotcv2mean, plotoversigmacv2, getovergenes, plotoverpoints
from tacos_plot import scatterdense

In [ ]:
import importlib, frontiers_analysis
importlib.reload(frontiers_analysis)
from frontiers_analysis import heaps, load_tissue, save_model, mazzolini

# Load Data

In [ ]:
data_source ="mca"
files = os.listdir(data_source)
from frontiers_analysis import load_all_data, clean_df
data = load_all_data(data_source)

In [ ]:
from frontiers_analysis import get_files, cleanup
#get_files()

In [ ]:
df_info = pd.read_excel("MCA_Figure2_Cell.Info.xlsx", index_col=0)

df_info['Tissue'] = list(map(lambda ann: ann.replace("Adult",""), df_info['Tissue'])) #remove adult or fetal
df_info['Tissue'] = list(map(lambda ann: ann.replace("Fetal",""), df_info['Tissue'])) #remove adult or fetal
df_info['Tissue'] = list(map(lambda ann: ann.replace("Neonatal",""), df_info['Tissue'])) #remove adult or fetal
df_info.info()

In [ ]:
df_file=pd.read_csv("MCA_CellAssignments.csv", index_col=0).set_index('Cell.name')
df_file['Cell_type'] = list(map(lambda ann: ann.split('(')[0].split(' ')[0], df_file['Annotation'])) #remove adult or fetal
print(df_file.info())
print(df_file['Tissue'].unique())
print(df_file['Cell_type'].unique())
df_file.head()

In [ ]:
df_info = df_file
df_info["id"] = df_file.index.values
df_info.rename(columns={'Tissue':'tissue',"id": "Cell_id"}, inplace=True)
df_info.info()

In [ ]:
files = list(filter(lambda f: "dge" in f,  os.listdir(data_source)))
files

# Analyse organs and store data

In [ ]:
print(files[0])

saved = []
frac_of = {}

if "mca" in data_source:
    data = pd.read_csv(f"{data_source}/{files[0]}", sep=" ")
    data = data[data.columns[data.columns.isin(df_info['Cell_id'])]]

    #tissues = df_info[df_info['Cell_id'].isin(data.columns) & (df_info['Development_stage']=='Adult')]['tissue'].unique() # only adults
    tissues = df_info[df_info['Cell_id'].isin(data.columns)]['tissue'].unique()

if "tm" in data_source:
    tissues = [files[0]]


print(tissues)
for tissue in tissues:
    if "mca" in data_source:
        df = data.reindex(columns=df_info[df_info['Cell_id'].isin(data.columns) & (df_info['tissue']==tissue)]['Cell_id'])
        #df = df.divide(df.sum(0), 1).applymap(lambda cpm: np.log(cpm*1e6+1)) #log(cpm +1)
    
    if "tm" in data_source:
        df = pd.read_csv(f"{data_source}/{files[0]}", sep=",", index_col=0) #tabula muris
        df = clean_df(df) #tabula muris
    
    A = df.sum(axis=1)
    O = df.apply(lambda x: len(x[x>0]), 1)
    M = df.apply(np.sum, 0)
    cell_zeros = df.astype(int).apply(lambda x: len(x[x==0]), 0)
    gene_presence = df.apply(lambda x: len(x[~x.isna()]), 1).astype(int)
    gene_presence_nonnull = df.apply(lambda x: len(x[~x.isna() & (x>0)]), 1).astype(int)
    var = df.apply(lambda x: (x*x).sum(), 1)
    
    frac_of[tissue] = {'data': A,
                            'N': len(M)}
    
    diffWords = df.apply(lambda x: len(x[x>0]), 0)
    n_genes = {}
    n_genes[tissue] = diffWords
    
    n_expressed_genes = pd.Series(index=df.index, data=np.zeros_like(df.index))
    n_expressed_genes[A.sort_values(ascending=False).index[:100]]+=1
    save_model(df, "data", tissue)
    #mazzolini(M, A/A.sum(), tissue)
    #null_model(df, M, A/gene_presence_nonnull)
    #heaps(M, diffWords, tissue)
    
    saved.append(tissue)
    df.to_csv(f"{data_source}/mainTable_{tissue}.csv", index=True, header=True)

In [ ]:
for file in files[1:]:
    print(file)
    if "mca" in data_source:
        data = pd.read_csv(f"{data_source}/{file}", sep=" ")
        data = data[data.columns[data.columns.isin(df_info['Cell_id'])]]

        if len(data.columns) < 1:
            print(f"skipping {file} \n")
            continue
        #tissues = df_info[df_info['Cell_id'].isin(data.columns) & (df_info['Development_stage']=='Adult')]['tissue'].unique() # only adults
        tissues = df_info[df_info['Cell_id'].isin(data.columns)]['tissue'].unique()

    if "tm" in data_source:
        data = pd.read_csv(f"{data_source}/{file}", sep=",", index_col=0)
        tissues = [files[0]]
    
    print(tissues)
    if len(tissues) < 1:
        print(f"no tissue found \n")
        continue
    for tissue in tissues:
        
        if "mca" in data_source:
            df = data.reindex(columns=df_info[df_info['Cell_id'].isin(data.columns) & (df_info['tissue']==tissue)]['Cell_id'])
        
        if "tm" in data_source:
            df = clean_df(data)
        
        if tissue in saved:
            df = df.transpose().append(pd.read_csv(f"{data_source}/mainTable_{tissue}.csv", index_col=0, header=0).transpose(), ignore_index=False, sort=True).transpose().fillna(0)
        
        #df = df.divide(df.sum(0), 1).applymap(lambda cpm: np.log(cpm*1e6+1)) #log(cpm +1)
        A_sub = df.sum(axis=1)
        O_sub = df.apply(lambda x: len(x[x>0]), 1)
        M_sub = df.apply(np.sum, 0)
        cell_zeros_sub = df.apply(lambda x: len(x[x>0]), 0)
        diffWords_sub = df.apply(lambda x: len(x[x>0]), 0)
        
        if tissue in frac_of.keys():
            frac_of[tissue]['data'].add(A_sub, fill_value = 0)
            frac_of[tissue]['N']+=len(M_sub)
        else:
            frac_of[tissue] = {'data': A_sub,
                                  'N': len(M_sub)}

        A = A.add(A_sub, fill_value=0)
        O = O.add(O_sub, fill_value=0)
        M = M.append(M_sub)
        cell_zeros = cell_zeros.append(cell_zeros_sub)
        diffWords = diffWords.append(diffWords_sub)
        if tissue in n_genes.keys():
            n_genes[tissue] = n_genes[tissue].append(diffWords_sub)
        else:
            n_genes[tissue]= diffWords_sub
        
        gene_presence = gene_presence.add(df.apply(lambda x: len(x[~x.isna()]), 1), fill_value=0)
        gene_presence_nonnull_sub = df.apply(lambda x: len(x[~x.isna() & (x>0)]), 1).astype(int)
        gene_presence_nonnull = gene_presence_nonnull.add(gene_presence_nonnull_sub, fill_value=0).astype(int)
        var = var.add(df.apply(lambda x: (x*x).sum(), 1), fill_value=0)
        
        n_expressed_genes_sub = pd.Series(index=df.index, data=np.zeros_like(df.index))
        n_expressed_genes_sub[A_sub.sort_values(ascending=False).index[:100]]+=1
        n_expressed_genes = n_expressed_genes.add(n_expressed_genes_sub, fill_value=0)

        save_model(df,"data",tissue)
        #mazzolini(M_sub, A_sub/A_sub.sum(), tissue)
        #null_model(df, M_sub, A_sub/gene_presence_nonnull_sub)
        #heaps(M_sub, diffWords_sub, tissue)
        saved.append(tissue)
        df.to_csv(f"{data_source}/mainTable_{tissue}.csv", index=True, header=True)
        print(f"Handling {len(M)} cells")

        del df 
        del A_sub
        del O_sub
        del M_sub
        del diffWords_sub
        del gene_presence_nonnull_sub
        gc.collect()
        print("\n")
    del tissues
    del data
    gc.collect()

means = A/gene_presence
means_nozero = A/gene_presence_nonnull
var = var/gene_presence - means*means
f = (A/A.sum())
O = O / gene_presence
O = O.reindex_like(means)
cv2 = var/means/means


print("Saving data.pkl")
data = {
    'means': means,
    'var': var,
    'freq': A/A.sum(),
    'O': O,
    'M': M,
    'cv2': cv2,
    'diffWords': diffWords,
    'means_nonzero': means_nozero,
    'n_expressed': n_expressed_genes,
    'n_genes': n_genes,
    'frac_of': frac_of,
    'cell_zeros': cell_zeros
}

with open("data_all.pkl","wb") as file:
    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
M=M[~M.duplicated()].astype(int)

In [ ]:
mazzolini(M,A/A.sum(),"Brain")

In [ ]:
n_expressed_genes = data["n_expressed"]
with open("100_most_expressed_in_40_tissues.txt", "w") as f:
    list(map(lambda x: f.write(x+"\n"),n_expressed_genes[n_expressed_genes>=40].sort_values(ascending=False).index[:100]))

In [ ]:
with open("genes_first_range_1_26_mca.txt", "w") as file:
    list(map(lambda g:file.write(g+"\n"),data["freq"].sort_values(ascending=False)[:26].index))

# Correlations with Tabula Muris

In [ ]:
from scipy.stats import pearsonr

In [ ]:
common_organs = ["Bladder", "Kidney", "Liver", "Lung", "Pancreas", "Spleen"]

organ = common_organs[0]

In [ ]:
# http://www.informatics.jax.org/marker
df_gene_name = pd.read_csv("MGImarkerQuery_20200914_050053.txt", sep="\t")
nc = df_gene_name[df_gene_name["Feature Type"]!="protein coding gene"]["Symbol"]

In [ ]:
tm = load_all_data("tm")["freq"]
mca = load_all_data("mca")["freq"]

organ="all_abundances"

#tm = load_tissue(f"{organ}-counts.csv", data_source="tm")["freq"]
#mca = load_tissue(f"{organ}", data_source="mca")["freq"]
#tm = tm / tm.sum()
#mca = mca / mca.sum()

tm = tm[tm.index.isin(mca.index)]
mca = mca[mca.index.isin(tm.index)]
tm = pd.Series(name='tm',index=tm.sort_values(ascending=False).index, data=range(1,1+len(tm)))
mca = pd.Series(name='mca',index=mca.sort_values(ascending=False).index, data=range(1, len(mca)+1))

mca = mca.reindex_like(tm).dropna()
tm = tm.reindex_like(mca).dropna()

tm_nc = tm[tm.index.isin(nc)]
mca_nc = mca[mca.index.isin(nc)]

In [ ]:
first_regime = pd.DataFrame()
first_regime.insert(0,"tm",list(filter(lambda g: "Olfr" not in g, tm.sort_values(ascending=False).index))[:49])
first_regime.insert(0,"mca",list(filter(lambda g: "Olfr" not in g, mca.sort_values(ascending=False).index))[:49])
first_regime.to_csv("common_first_regime_noOlfr.csv", index=False, header=True)

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))
ax.scatter(mca.values,tm.values, c="gray", s=225, alpha=0.5)
#scatterdense(mca.fillna(0).astype(float),tm.fillna(0).astype(float), ax=ax)

#ax.scatter(mca_nc.values,tm_nc.values, c="red", s=225, alpha=0.5)

bin_avg, bin_edges, _ = stats.binned_statistic(mca.values, tm.values,"mean", bins=np.linspace(1,2e4,20))
ax.plot(np.linspace(0,len(mca)),np.linspace(0,len(mca)), lw=15, ls="--", label="diagonal")
ax.hlines(bin_avg, bin_edges[:-1], bin_edges[1:], lw=15, ls="-", color="orange", label="binned average")


ax.set_title(f"{organ} (Pearson = %.2f)"%(pearsonr(mca.values, tm.values)[0]), fontsize=35)
ax.set_xlabel("Mouse Cell Atlas' Rank", fontsize=35)
ax.set_ylabel("Tabula Muris' Rank", fontsize=35)

#ax.set_xscale("log")
#ax.set_yscale("log")
ax.set_xlim(mca.min(),mca.max())
ax.set_ylim(tm.min(),tm.max())

ax.tick_params(labelsize=35, width=8, length=20)
ax.legend(loc="lower right", fontsize=35)
fig.savefig(f"rank_correlation_{organ}.pdf")
fig.savefig(f"rank_correlation_{organ}.eps")
plt.show()

In [ ]:
df_both = pd.DataFrame()
df_both=df_both.append(mca)
df_both=df_both.append(tm)
df_both=df_both.transpose()

pop = df_both.index
sampled=df_both.sort_values(by="mca", ascending=True)[:500].index
successes = df_both.sort_values(by="tm", ascending=True)[:500].index

x = np.isin(successes, sampled).astype(int).sum() # number of successes
M = len(pop) # pop size
k = len(successes) # successes in pop
N = len(sampled) # sample size
pval = hypergeom.sf(x-1, M, k, N)
#-np.log10(float(pval))
pval

In [ ]:
df_cmap=pd.DataFrame(index=common_organs, columns=common_organs).fillna(0.5)
genes_list = {}
lim = 18
d_lim = 2000

rang=range(0,82)

mca = load_tissue(organ)["freq"]
tm = load_tissue(organ+"-counts.csv",  data_source="tm")["freq"]
tm_all = tm[tm.index.isin(mca.index)]
mca_all = mca[mca.index.isin(tm.index)]
common_genes = mca_all.index

population_size = len(mca_all)


for tissue_1 in df_cmap.columns:
    genes_list["tm"]=load_tissue(tissue_1+"-counts.csv", data_source="tm")["freq"]
    for tissue_2 in df_cmap.index:
        print(tissue_1, tissue_2)
        genes_list["mca"]=load_tissue(tissue_2)["freq"]
        tm = pd.Series(name='tm',index=genes_list["tm"].sort_values(ascending=True).index, data=range(1,1+len(genes_list["tm"])))
        mca = pd.Series(name='mca',index=genes_list["mca"].sort_values(ascending=True).index, data=range(1, len(genes_list["mca"])+1))
        tm = tm[tm.index.isin(common_genes)]
        mca = mca[mca.index.isin(common_genes)]

        x = np.isin(tm.index[rang], mca.index[rang]).astype(int).sum() # number of successes
        M = population_size # pop size
        k = len(mca.index[rang]) # successes in pop
        N = len(tm.index[rang]) # sample size
        pval = hypergeom.sf(x-1, M, k, N)
        print(x, M, k, N, pval)
        log_pval = -np.log10(float(pval))
        if log_pval < -np.log10(5e-1):
            log_pval=0
        df_cmap.at[tissue_1,tissue_2]=log_pval

In [ ]:
ax = sns.heatmap(df_cmap)
ax.set_title("-Log(P-value)\n Tabula Muris vs Mouse Cell Atlas", fontsize=25)

fig= ax.get_figure()
fig.set_size_inches(18,15)
fig.savefig("logp_regim1.pdf")

In [ ]:
from frontiers_analysis import load_pickle
mca_genes = load_pickle("mca/data_all.pkl")["freq"].index

In [ ]:
bad_genes = pd.read_csv("FACS/Aorta-counts.csv", index_col=0).index[np.load("bad_genes_0.npy")]
isin = np.isin(mca_genes,bad_genes)
with open("bad_genes_common.csv", "w") as file:
    list(map(lambda g: file.write(g+"\n"),mca_genes[isin]))

In [ ]:
ax = sns.heatmap(pd.read_csv("rs_gtex.csv"))

# Fraction of trascriptome

In [ ]:
thresholded = {}

frac_of = data["frac_of"]

fig,ax = plt.subplots(figsize=(18,15))
for tissue in frac_of.keys():
    #if "Lactation" in tissue:
    #    continue
    A_tissue = (frac_of[tissue]['data'] / frac_of[tissue]['N']).sort_values(ascending=False)
    A_tissue_sum = np.cumsum(A_tissue.values) / A_tissue.sum() #h thr
    ax.plot(A_tissue_sum, label = tissue, c=get_color(tissue), lw=15, alpha=.7)
    #if len(A_tissue_sum) < 2:
    #    continue
    thresholded[tissue] = A_tissue_sum[99]
    #thresholded[tissue] = np.argwhere(A_tissue_sum>=0.4).ravel()[0] #v thr
ax.set_xscale('log')
ax.set_xlim(1, 3e4)
ax.set_xlabel("Number of genes", fontsize=35)
ax.set_ylabel("Fraction of trascriptome described", fontsize=35)

# Shrink current axis by 20%
box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

ax.tick_params(labelsize=35, width=8, length=20)

# Put a legend to the right of the current axis
#ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()
fig.savefig("fracof_mca.pdf")

In [ ]:
gene_to_thr = pd.DataFrame(data= thresholded.items()).set_index(0).sort_values(1, ascending=False)

fig,ax = plt.subplots(figsize=(20,10))
ax.bar(gene_to_thr.index, gene_to_thr.values.ravel())
ax.tick_params(rotation=90)

ax.set_ylabel("Fraction of trascriptome\n described by 100 genes", fontsize=24)
#ax.set_ylabel("Number of genes\n to describe 40% of trascriptome", fontsize=24)

plt.tight_layout()
plt.show()
fig.savefig("frac_of_100genes.pdf")

In [ ]:
pd.DataFrame(index=gene_to_thr.sort_values(1).index, data=range(len(gene_to_thr)), columns=['MCA']).to_csv("trascriptome_sort.csv", index=True)

In [ ]:
df_sort = pd.read_csv("trascriptome_sort.csv", index_col=0).dropna(how='any', axis=0).astype(int)
df_sort['MCA'] = range(len(df_sort))
fig = plt.figure(figsize=(10,8))
plt.scatter(*df_sort.values.T, s=90)
plt.xlim(-1,15)
plt.ylim(-1,15)

plt.xlabel("MCA", fontsize=20)
plt.ylabel("tabula_muris", fontsize=20)
plt.show()

In [ ]:
for g in frac_of['MammaryGland.Lactation']['data'].sort_values(ascending=False).index[:10]:
    print(g)

## Box plot

In [ ]:
fig,ax=plt.subplots(figsize=(45,35))
organ_names=n_genes.keys()
n_genes_all=list(map(lambda organ: organ[1].values,n_genes.items()))
box1=plt.boxplot(np.array(n_genes_all), vert=True, patch_artist=True,notch=True,
        boxprops=dict(facecolor='magenta', color='magenta'))
ax.set_xticklabels(labels=organ_names,rotation=75)
ax.tick_params(axis='y', labelsize=24)
ax.tick_params(axis='x', labelsize=18)
plt.title('Number of genes detected per cell', fontsize=28)
plt.xlabel('Organ', fontsize=28)
plt.ylabel('N genes', fontsize=28)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.show()
fig.savefig("boxplot.pdf")

# All Tissues

## Zipf

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))
for tissue in data["frac_of"].keys():
    try:
        info = load_tissue(tissue, data_source=data_source)
    except:
        print(*sys.exc_info())
        continue 
    if "Lactation" in tissue:
        continue
    f = info["freq"]
    f = f[~f.index.isin(nc)]
    ax.plot(f.sort_values(ascending=False).values, label=tissue, c=get_color(tissue), lw=15, alpha=.7)
    
x =np.linspace(1,6e4,10)

exp = 0.8
ax.plot(x, 1e-1*np.power(x, -0.8), lw=20, c='gray', ls='--')

ax.set_xscale("log")
ax.set_yscale("log")

ax.annotate('$k*i^{-%.1f}$'%exp, (1e2,1e-2), fontsize=35)

# Shrink current axis by 20%
box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
#ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=18, ncol=2)

ax.tick_params(labelsize=35, width=8, length=20)
ax.set_xlim(1,1e5)
ax.set_ylim(1e-9,1e0)
ax.set_xlabel("Rank, $i$", fontsize=35)
ax.set_ylabel("Frequency, $f_i$", fontsize=35)

plt.show()
fig.savefig(f"zipf_alltissue_{data_source}.pdf")

## Heaps

In [ ]:
loglog = True
filter_mammaryLactation = True

fig, ax = plt.subplots(figsize=(18,15))
for tissue in data["frac_of"].keys():
    try:
        info = load_tissue(tissue, data_source=data_source)
    except:
        print(*sys.exc_info())
        continue
    if ("Lactation" in tissue) and filter_mammaryLactation:
        continue
    M = info["M"]
    diff= info["diffWords"]
    skip_bins = 5
    n_bins=35
    if len(M) <= n_bins:
        continue
    bin_means, bin_edges, binnumber = stats.binned_statistic(M, diff,statistic='mean', bins=np.logspace(np.log10(M.min()),np.log10(max(M)), n_bins))
    bin_stds, _, _ = stats.binned_statistic(M, diff,statistic='std', bins=np.linspace(M.min(),np.quantile(M, 0.9), n_bins))
    x = ((bin_edges[:-1]+bin_edges[1:])/2.)[:-skip_bins]
    y = bin_means[:-skip_bins]
    ax.plot(x, y, marker='o', lw=5, ms=25, alpha=0.8, c=get_color(tissue), label=tissue)
    #ax.errorbar(x,y , bin_stds[:-skip_bins], fmt='none', ecolor='orange', elinewidth=3)

if "mca" in data_source:
    ax.set_xlabel("UMI per cell", fontsize=35)
else:
    ax.set_xlabel("Reads per cell", fontsize=35)
ax.set_ylabel("Number of expressed genes", fontsize=35)

if loglog:
    ax.set_xscale('log')
    ax.set_yscale('log')
ax.set_xlim(450,max(data["M"])*0.7) #min UMI is 500

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=25)


# Shrink current axis by 20%
box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
#ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=18, ncol=2)

plt.tight_layout()
plt.show()
fig.savefig("heaps_alltissues%s_mca%s.pdf"%("" if filter_mammaryLactation else "_noLactation","_loglog" if loglog else ""))

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))

ax.hist(data["M"], density=True, color="gray", histtype="step", lw=15, bins=50)

m = np.mean(data["M"])
ax.vlines([m], 0,2e-8, lw=10, alpha=.9, ls="-.")
ax.annotate("Average: %.2e"%m, (m*1.1,2e-8), fontsize=35)

ax.set_xlabel("UMI per cell, $M$", fontsize=35)
ax.set_ylabel("Probability density function", fontsize=35)

#ax.set_xscale("log")
#ax.set_yscale("log")

#ax.set_xlim(1,3e8)

plt.show()
fig.savefig(f"M_hist_{data_source}.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
for tissue in data["frac_of"].keys():
    try:
        info = load_tissue(tissue)
    except:
        continue
    M = info["M"]
    diff= info["diffWords"]
    x = M
    y = diff
    ax.plot(x, y, marker='o', lw=0, ms=10, alpha=0.6, label=tissue)
    #ax.errorbar(x,y , bin_stds[:-skip_bins], fmt='none', ecolor='orange', elinewidth=3)

    
ax.set_xlabel("Realization size", fontsize=24)
ax.set_ylabel("#different words", fontsize=24)

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=25)

#plt.xscale('log')
#plt.yscale('log')
plt.xlim(0,max(data["M"])*0.7)
    
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=18, ncol=2)

plt.show()
fig.savefig("heaps_alltissues_alldata.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

M = data["M"]
diff= data["diffWords"]
x = M
y = diff
ax.plot(x, y, marker='o', lw=0, ms=10, alpha=0.6, label=tissue)
#ax.errorbar(x,y , bin_stds[:-skip_bins], fmt='none', ecolor='orange', elinewidth=3)
#scatterdense(x,y, ax= ax)
    

ax.set_xlabel("Realization size", fontsize=24)
ax.set_ylabel("#different words", fontsize=24)

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=25)

#plt.xscale('log')
#plt.yscale('log')
plt.xlim(0,max(data["M"])*0.7)
    
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

plt.show()
fig.savefig("heaps_alltissues.pdf")

In [ ]:
from scipy.optimize import curve_fit
def heaps_taylor(X, k):
    return k * np.power(X, 2)

def heaps_free_exponent(X, C, k):
    return C * np.power(X, k)

In [ ]:
loglog = True
filter_mammaryLactation = True

exps = []

fig, ax = plt.subplots(figsize=(36,15))
for tissue in data["frac_of"].keys():
#for tissue in common_organs:
    try:
        info = load_tissue(tissue)
    except:
        continue
    if ("Lactation" in tissue) and filter_mammaryLactation:
        continue
    #data = load_all_data("mca")
    M = info["M"]
    diff= info["diffWords"]
    n_bins=25

    bins = np.logspace(np.log10(M.min()),np.log10(M.max()), n_bins)
    #bins = np.linspace(M.min(),max(M), n_bins)
    #bins = np.quantile(M,np.linspace(0,1, n_bins))



    bin_means, bin_edges, binnumber = stats.binned_statistic(M, diff,statistic='mean', bins=bins)
    bin_stds, _, _ = stats.binned_statistic(M, diff,statistic='std', bins=bins)
    bin_counts, _, _ = stats.binned_statistic(M, diff,statistic='count', bins=bins)

    bin_stds = np.power(bin_stds,2)

    mask = bin_counts > 1e2
    
    x = bin_means[mask]
    y = bin_stds[mask]
    popt = [1]
    try:
        popt, pcov = curve_fit(heaps_free_exponent, x,y)
        print(tissue, *popt)
        exps.append(popt[1])
    except:
        print(*sys.exc_info())
        print(tissue)
    finally:
        ax.plot(x, y, marker='o', lw=5, ms=25, alpha=0.6, c=get_color(tissue))

x = np.linspace(100,5e3)
ax.plot(x,x, lw=15,ls="--", c="blue", label="Poisson")
ax.plot(x,x*x, lw =15, ls="--", c="red", label="Taylor")
ax.set_xlabel("Mean number of expressed genes", fontsize=35)
ax.set_ylabel("Variance number of expressed genes", fontsize=35)

ax.set_xlim(200,5e3) #min UMI is 500

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=25)

ax.set_xscale('log')
ax.set_yscale('log')

# Shrink current axis by 20%
box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(fontsize=35, ncol=1)

plt.tight_layout()
plt.show()
fig.savefig("heaps_alltissues%s_mca%s_fluctuaction.pdf"%("" if filter_mammaryLactation else "_noLactation","_loglog" if loglog else ""))

In [ ]:
fig,ax=plt.subplots(figsize=(36,15))

n_bins = 10
bins = np.linspace(0-5/2/n_bins,5+5/2/n_bins,n_bins+2)
ax.hist(exps, histtype="step", density=True, color="gray", lw=15, bins=bins)

ax.set_xlabel("Fit's exponent", fontsize=35)
ax.set_ylabel("pdf", fontsize=35)
ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=25)
plt.show()
fig.savefig("heaps_alltissues%s_mca%s_fluctuaction_exponent_hist.pdf"%("" if filter_mammaryLactation else "_noLactation","_loglog" if loglog else ""))

In [ ]:
loglog = True
filter_mammaryLactation = True

fig, ax = plt.subplots(figsize=(18,15))
info = load_all_data("mca")
M = info["M"]
diff= info["diffWords"]
n_bins=35

bins = np.logspace(np.log10(M.min()),np.log10(M.max()), n_bins)
#bins = np.linspace(M.min(),max(M), n_bins)
#bins = np.quantile(M,np.linspace(0,1, n_bins))

bin_means, bin_edges, binnumber = stats.binned_statistic(M, diff,statistic='median', bins=bins)
bin_stds, _, _ = stats.binned_statistic(M, diff,statistic='std', bins=bins)
bin_counts, _, _ = stats.binned_statistic(M, diff,statistic='count', bins=bins)

strong_mask = bin_counts > 5e3
mask = bin_counts > 1e2

bin_stds = np.power(bin_stds,2)

x = bin_means
y = bin_stds

popt, pcov = curve_fit(heapsvar, x[strong_mask], y[strong_mask])
print(*popt)
print(pcov)

ax.plot(x[strong_mask], y[strong_mask], marker='o', lw=15, ms=35, alpha=0.6, c="gray")
ax.plot(x[strong_mask], heapsvar(x[strong_mask], *popt), lw=8, ms=25, alpha=0.8, c="red", label="Taylor times constant fit")

#ax.errorbar(x,y , bin_stds[:-skip_bins], fmt='none', ecolor='orange', elinewidth=3)

#popt, pcov = curve_fit(lambda x, C: C*x, x[(~strong_mask)*mask], y[(~strong_mask)*mask])
#print(*popt)
#print(pcov)
#ax.plot(x[~strong_mask*mask], popt[0]*x[~strong_mask*mask], lw=8, ms=25, alpha=0.8, c="blue")


ax.plot(x,x, lw=15,ls="--", c="blue", label="Poisson")
ax.plot(x,x*x, lw =15, ls="--", c="red", label="Taylor")
ax.set_xlabel("Mean number of expressed genes", fontsize=35)
ax.set_ylabel("Variance number of expressed genes", fontsize=35)

ax.set_xlim(300,2e3) #min UMI is 500

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=25)

ax.set_xscale('log')
ax.set_yscale('log')

# Shrink current axis by 20%
box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(fontsize=35, ncol=1)

plt.tight_layout()
plt.show()
fig.savefig("heaps_alltissues%s_mca%s_fluctuaction_all.pdf"%("" if filter_mammaryLactation else "_noLactation","_loglog" if loglog else ""))

In [ ]:
integrals = pd.read_csv("integral_heaps.csv", index_col=1)
integrals=integrals.sort_values("data", ascending=False)
integrals.head()

In [ ]:
fig = plt.figure(figsize=(18,10))
for x,tissue in enumerate(integrals.index):
    plt.plot(x, integrals.at[tissue,"data"], marker="P", c=get_color(tissue), lw=0, ms=15)
locs, labels = plt.xticks()
plt.xticks(range(0,len(integrals.index)), labels = integrals.index, rotation=90)
plt.ylabel("Heaps' integral", fontsize=20)
fig.savefig("itegral_heapfs_scatter_plot_color_mca.pdf")
fig.savefig("itegral_heapfs_scatter_plot_color_mca.png")

# Null Model

## U

In [ ]:
def p(o, M, gamma, N):
    i = np.arange(1, N+1, step=1)
    alpha_i = i**(-gamma)
    alpha = np.sum(alpha_i, axis=0)
    p_num = (1-o)**(1/M-1)
    K = (1-(1-o)**(1/M))**(1+1/gamma)
    p_den = gamma*M*N*(alpha**(1/gamma))*K
    return p_num/p_den

In [ ]:
nbins = 10
rang = (0-0.5/nbins, 1+0.5/nbins)

bins = np.logspace(np.log10(1e-2),np.log10(rang[1]),num=nbins)
#bins = np.linspace(rang[0], rang[1], num=nbins)

fig, ax =plt.subplots(figsize=(18,15))
ax.hist(np.array(O, dtype=float), histtype='step', lw=15, color='gray', density=True, bins=bins)
if "O_null" in vars().keys():
    ax.hist(np.array(O_null, dtype=float), histtype='step', lw=15, bins=bins, color='red', ls='--', density=True)
ax.set_xlabel("Occurrence, $o_i$", fontsize=35)
ax.set_ylabel("Probability Density Function", fontsize=35)

x_bins=np.arange(1e-2,rang[1],step=(rang[1]-rang[0])/nbins)
ax.plot(x_bins,[p(x, M.mean(), 0.8, len(O)) for x in x_bins], lw=15, ls="--", c="orange")

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=15)

ax.set_xlim(9e-3,1.5)

ax.set_yscale('log')
ax.set_xscale('log')
plt.show()
fig.savefig(f"U_{data_source}.pdf")

In [ ]:
fig = plt.figure()
plt.hist(M, density=False, color='blue', bins = 15, label='files')
#plt.hist(M_null, density=False, color='orange', histtype='step', lw=4, ls='--', bins = 15, label='files')
plt.title("realization size distribution", fontsize = 24)
plt.xlabel("total counts per file", fontsize=24)
plt.ylabel("#", fontsize=24)
plt.legend(fontsize=24)
plt.show()
fig.savefig("M_distr.png")

In [ ]:
fig = plt.figure()
plt.hist(diffWords, density=False, color='blue', bins = 15, label = 'files')
plt.title("vocabulary size distribution", fontsize=18)
plt.xlabel("#different words per file", fontsize=16)
plt.ylabel("#", fontsize=16)
plt.legend(fontsize=18)
plt.show()
fig.savefig("gene_expr_distr.png")

## Heaps

In [ ]:
def h(m, gamma, N):
    i = np.arange(1, N+1, step=1)
    alpha_i = i**(-gamma)
    alpha = np.sum(alpha_i, axis=0)
    return N - np.sum((1-np.power(i, -gamma)/alpha)**m, axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))


ax.scatter(data["M"], data["diffWords"], label='samples', alpha=0.5, s=225, c="gray")
if "M_null" in vars().keys() and "diffWords_null" in vars().keys():
    plt.scatter(M_null, diffWords_null, label='null_model', s=225, c="red")
ax.set_ylabel("Number of genes expressed", fontsize=35)
ax.set_xlabel("UMI per cell", fontsize=35)
skip_bins = 2
n_bins=15

bin_means, bin_edges, binnumber = stats.binned_statistic(data["M"], data["diffWords"],statistic='mean', bins=np.logspace(np.log10(data["M"].min()),np.log10(max(data["M"])), n_bins))
#ax.hlines(bin_means[:-skip_bins], bin_edges[:-1][:-skip_bins], bin_edges[1:][:-skip_bins], colors='r', lw=5, label='binned average')
#bin_stds, _, _ = stats.binned_statistic(data["M"], data["diffWords"],statistic='std', bins=np.linspace(M.min(),np.quantile(M, 0.9), n_bins))
#ax.errorbar(((bin_edges[:-1]+bin_edges[1:])/2)[:-skip_bins],bin_means[:-skip_bins], bin_stds[:-skip_bins], fmt='none', ecolor='orange', elinewidth=3)

ax.plot((bin_edges[:-1][:-skip_bins] + bin_edges[1:][:-skip_bins])/2, bin_means[:-skip_bins], color='blue', lw=10, marker="o", ms=25, label='binned average')


x_bins=np.logspace(np.log10(data["M"].min()),np.log10(max(data["M"])))
ax.plot(x_bins, [h(x, 0.8, len(data["freq"].dropna())) for x in x_bins], lw=15, ls="--", c="orange", label="model")

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(450,max(M)*1.1)

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=15)

ax.legend(fontsize=35)
plt.show()
fig.savefig("heaps.pdf")

In [ ]:
fit_func = lambda x, a, b: a + np.power(x, b)
from scipy.optimize import curve_fit

bin_vars = bin_stds*bin_stds
fig, ax = plt.subplots(figsize=(10,6))
ax.scatter(bin_means[:-skip_bins], bin_vars[:-skip_bins],s=90, label='')
ax.plot(bin_means[:-skip_bins], bin_means[:-skip_bins], label='Poisson')
ax.plot(bin_means[:-skip_bins], bin_means[:-skip_bins]**2, label='Taylor')

popt, pcov = curve_fit(fit_func, bin_means[:-skip_bins], bin_vars[:-skip_bins])
ax.plot(bin_means[:-skip_bins], fit_func(bin_means[:-skip_bins], *popt), label="%.0f + x**%.1f"%(popt[0],popt[1]))

ax.set_xlabel("Heaps' mean", fontsize=24)
ax.set_ylabel("Heaps' var", fontsize=24)
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(fontsize=24)
#ax.set_ylim(1e3,3e4)
ax.set_xlim(np.nanmin(bin_means),np.nanmax(bin_means))
plt.show()
fig.savefig("heaps_Taylor.pdf")

In [ ]:
from scipy.integrate import quad

In [ ]:
fit_bins = lambda x, a, b: a + np.power(x,b)

fig = plt.figure(figsize=(10,6))
plt.scatter(data["M"], diffWords, label='samples')
plt.xlabel("Realization size", fontsize=24)
plt.ylabel("#different words", fontsize=24)
n_bins=35
bin_means, bin_edges, binnumber = stats.binned_statistic(data["M"], diffWords,statistic='mean', bins=np.linspace(data["M"].min(),max(data["M"]), n_bins))
bin_counts, bin_edges, binnumber = stats.binned_statistic(data["M"], diffWords,statistic='count', bins=np.linspace(data["M"].min(),max(data["M"]), n_bins))

skip_bins=(bin_counts<10).astype(int).sum()
x_bins = ((bin_edges[:-1]+bin_edges[1:])/2)[:-skip_bins]

plt.hlines(bin_means[:-skip_bins], bin_edges[:-1][:-skip_bins], bin_edges[1:][:-skip_bins], colors='r', lw=5, label='binned average')
bin_stds, _, _ = stats.binned_statistic(data["M"], diffWords,statistic='std', bins=np.linspace(M.min(),np.quantile(M, 0.9), n_bins))
plt.errorbar(x_bins,bin_means[:-skip_bins], bin_stds[:-skip_bins], fmt='none', ecolor='orange', elinewidth=3)

popt, pcov = curve_fit(fit_bins, x_bins, bin_means[:-skip_bins])

plt.plot(np.linspace(500, 2e4), fit_bins(np.linspace(500, 2e4), *popt), lw=4, c='cyan')

#plt.xscale('log')
#plt.yscale('log')
plt.xlim(0,max(data["M"])*1.1)
plt.legend(fontsize=20)
plt.show()
fig.savefig("heaps.pdf")

# Scaling

## CV2

In [ ]:
var = data["var"].dropna()
means = data["means"].dropna()
O = data["O"].dropna()
O = O[~O.duplicated()]
means = means[~means.duplicated()]
var = var[~var.duplicated()]
means = means[var[var>1e-7].index].dropna()
var = var.reindex_like(means)
O = O.reindex_like(means)

In [ ]:
df_genes=pd.read_csv("genes_gtex.txt", header=0, index_col=0)
df_genes.head()
nc=df_genes[df_genes["type_of_gene"]!="protein-coding"].index
pc=df_genes[df_genes["type_of_gene"]=="protein-coding"].index
genelist = pd.read_csv("https://stephenslab.github.io/count-clustering/project/utilities/gene_names_all_gtex.txt", header=None).values.ravel()
pc=pc[pc.isin(genelist)]

In [ ]:
#select protein coding or noncoding
means = means[means.index.isin(pc)]
var = var.reindex_like(means)

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))
ax.set_xlim(1e-3, 5e2)

plotcv2mean(means.values.ravel(), var.values.ravel(), ax=ax, normalisation_str='counts', poisson_limit=1, alpha=0.5, s=255, colorbar=True)

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(axis="both", which="minor", width=5, length=10)

upbound = len(data["M"])-1
x = np.logspace(-5,7)
ax.plot(x,[upbound for _ in x], ls="--", lw=15, c="orange", label="R-1")

ax.legend(fontsize=30)
ax.set_ylim(5e-1,5e5)
plt.show()
fig.savefig(f"cv2_mean_dense_{data_source}.pdf")
fig.savefig(f"cv2_mean_dense_{data_source}.png", dpi=400)

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

fig, ax = plt.subplots(figsize=(18,15))

#plt.scatter(means, var, c='gray', alpha=0.3, s=80, label='genes')
plotvarmen(means.values, var.values, ax=ax, alpha=0.5, s=255, colorbar=True)

ax.tick_params(labelsize=35, width=8, length=20)
ax.set_ylim(1e-6,np.power(10,np.log10(var.max())+1))
ax.set_xlim(1e-5,max(means)*1.5)
ax.legend(fontsize=35)
plt.show()
fig.savefig(f"var_mean_dense_{data_source}.pdf")
fig.savefig(f"var_mean_dense_{data_source}.png", dpi=400)

# Single cell zipf

In [ ]:
df = pd.read_csv("mca/mainTable_Bone-Marrow_c-kit.csv", header=0, index_col=0)
M = df.sum(0)

In [ ]:
top100 = M.sort_values(ascending=False).index[:100].values

In [ ]:
np.save("top100_cells_bm.npy", df[df.columns[df.columns.isin(top100)]].to_numpy())

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))
df[df.columns[df.columns.isin(top100)]].apply(lambda x: ax.plot((x.sort_values(ascending=False)/x.sum()).values), axis=0)

x =np.linspace(1,6e3,10)

exp = 0.8
ax.plot(x, 1e-1*np.power(x, -0.8), lw=20, c='gray', ls='--')
ax.annotate('$k*i^{-%.1f}$'%exp, (1e2,1e-2), fontsize=35)


ax.set_ylabel("$Frequency, f_i$", fontsize=35)
ax.set_xlabel("$RANK, i$", fontsize=35)

ax.set_xscale("log")
ax.set_yscale("log")

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", width=5, length=15)


ax.set_xlim(1,2e4)
ax.set_ylim(6e-5,1e-1)

plt.show()
fig.savefig("zipf_top100_mca.pdf")
fig.savefig("zipf_top100_mca.png", dpi=400)



# Mean vs Occurrence

In [ ]:
fig=plt.figure(figsize=(10,8))
nfiles = len(data["M"])
plt.scatter(O*nfiles, means, c='b', alpha=0.8, label='genes')
#plt.scatter(O_null*len(M_null), means_null*5, c='orange', alpha=0.8, label='genes')

x = np.linspace(1,nfiles)
plt.plot(x, x/(nfiles), lw=4, label='bound', c='cyan', ls='--')


bin_means, bin_edges, _ = stats.binned_statistic(O*nfiles, means, statistic='mean', bins=np.logspace(-3,6))
x = (bin_edges[1:]+bin_edges[:-1])/2
plt.scatter(x,bin_means, marker='x', c='r', label='binned average')
plt.ylabel("$<counts>$", fontsize=16)
plt.xlabel("$\Sigma_j\Theta(counts)$", fontsize=16)
plt.xscale('log')
plt.yscale('log')
plt.ylim(means[means!=0].min()/5,np.power(10,np.log10(means.max())+1))
plt.xlim(5e-1,nfiles+8000)
plt.legend(fontsize=20)
plt.show()
fig.savefig("mean_occ.pdf")

In [ ]:
from scipy.optimize import curve_fit

fig=plt.figure(figsize=(12,8))
nfiles = len(data["M"])
means_nozero = means_nozero.dropna()
x_data = O.reindex_like(means_nozero)*nfiles

plt.scatter(x_data, means_nozero, c='b', alpha=0.8, label='genes')

bin_means, bin_edges, _ = stats.binned_statistic(x_data, means_nozero, statistic='mean', bins=np.logspace(0,5,30))
x = (bin_edges[1:]+bin_edges[:-1])/2.
plt.scatter(x,bin_means, marker='x', c='r', label='binned average')

bin_mins, _, _ = stats.binned_statistic(x_data, means_nozero, statistic='min', bins=np.logspace(0,5,30))
#f = lambda x, a, b, c, d:np.exp(-a*x) + b*x + c + d*x*x
#f = lambda x, a, C: C + np.power(a, -x)

#popt, pcov = curve_fit(f, x[2:], bin_mins[2:] )
#plt.scatter(x,bin_mins, marker='x', c='g', label='bound')
#plt.plot(x,f(x, *popt), c='g', label='bound')


plt.ylabel("$<counts(>0)>$", fontsize=16)
plt.xlabel("$\Sigma_j\Theta(counts)$", fontsize=16)
plt.xscale('log')
plt.yscale('log')
plt.ylim(0.5,np.power(10,np.log10(means.max())+1))
plt.xlim(5e-1,nfiles+10000)
plt.legend(fontsize=20)
plt.show()
fig.savefig("mean_occ_non_zero.pdf")

# M vs N_zeros

In [ ]:
from scipy.optimize import curve_fit

x_data = M
y_data = cell_zeros/len(A)

plt.scatter(x_data, y_data, label='cell')
bin_means, bin_edges, _ = stats.binned_statistic(x_data, y_data, statistic='mean', bins=np.linspace(0,8000,10))
x = ((bin_edges[1:]+bin_edges[:-1])/2.)[~np.isnan(bin_means)]
bin_means = bin_means[~np.isnan(bin_means)]
plt.scatter(x,bin_means, marker='x', c='r', label='binned average')

fit_func = lambda x, a, C: C + a*x


popt, pcov = curve_fit(fit_func, x, bin_means )
plt.plot(x,fit_func(x, *popt), c='g', ls='--', label='fit (%.10f x + %.3f)'%(popt[0], popt[1]))

plt.xlabel("M")
plt.ylabel("#zeri/#geni")

plt.legend()
plt.show()

In [ ]:
from scipy.optimize import curve_fit

x_data = M
y_data = np.log10(cell_zeros/len(A))

plt.scatter(x_data, y_data, label='cell')
bin_means, bin_edges, _ = stats.binned_statistic(x_data, y_data, statistic='mean', bins=np.linspace(0,4000,10))
x = ((bin_edges[1:]+bin_edges[:-1])/2.)[~np.isnan(bin_means)]
bin_means = bin_means[~np.isnan(bin_means)]
plt.scatter(x,bin_means, marker='x', c='r', label='binned average')

fit_func = lambda x, a, C: C + a*x


popt, pcov = curve_fit(fit_func, x, bin_means )
plt.plot(x,fit_func(x, *popt), c='g', ls='--', label='fit (%.5f x + %.3f)'%(popt[0], popt[1]))

plt.xlabel("M")
plt.ylabel("log(#zeri/#geni)|M")
plt.xlim(0,8000)

plt.legend()
plt.show()

In [ ]:
from scipy.optimize import curve_fit

x_data = M
y_data =cell_zeros/len(A)

plt.scatter(x_data, np.log2(y_data), label='cell')
bin_means, bin_edges, _ = stats.binned_statistic(x_data, y_data, statistic='mean', bins=np.linspace(0,4000,10))
x = ((bin_edges[1:]+bin_edges[:-1])/2.)[~np.isnan(bin_means)]
bin_means = bin_means[~np.isnan(bin_means)]
plt.scatter(x,np.log2(bin_means), marker='x', c='r', label='binned average')

fit_func = lambda x, a, C: C + a*x


popt, pcov = curve_fit(fit_func, x, bin_means )
plt.plot(x,np.log2(fit_func(x, *popt)), c='g', ls='--', label='fit (%.5f x + %.3f)'%(popt[0], popt[1]))

plt.xlabel("M")
plt.ylabel("log(#zeri/#geni|M)")

plt.xlim(0,8000)

plt.legend()
plt.show()

# P(0|M)

In [ ]:
def p0m(df, M, limits = (100,110)):
    M_sorted = M.sort_values(ascending=True)
    quantiles = np.quantile(M, q=np.linspace(0,1,10)[:-1])
    #quantiles = np.linspace(M.min(), M.max(), 20)
    M_classes = pd.Series(index=M.index, data=np.digitize(M, quantiles))
    f = df.mean(1)
    f = f/f.sum()
    f_ = df.apply(lambda x: x[x>0].mean(), 1)
    f_ = f_/f_.sum()
    ret = {}
    for g in f.sort_values(ascending=False).index[limits[0]:limits[1]]:
        genexpr = df.loc[g,:]
        ret[g] =  [(genexpr[M_classes[M_classes==c].index]==0).astype(int).sum()/len(M_classes[M_classes==c]) for c in np.arange(len(quantiles))+1]
        del genexpr
        gc.collect()
    return ret, quantiles, (f.sort_values(ascending=False)[limits[0]:limits[1]].mean(), f_.sort_values(ascending=False)[limits[0]:limits[1]].mean())

In [ ]:
from scipy.optimize import curve_fit
import json
fig = plt.figure(figsize=(20,10))
palette = itertools.cycle(sns.color_palette(palette="Set1", n_colors=100, desat=.9))

def markers():
    for m in ['x', 'o', '.', ',', 'v', '>', '<', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
        yield m

subdf = pd.read_csv("mca/mainTable_Stomach.csv", index_col=0)
M = subdf.sum(0)
results = pd.DataFrame(columns=["rank", "a", "b", "f", "f_"])
for under_limit in np.linspace(100, 2e4, num=25, dtype=int):
    print(under_limit)
    color = next(palette)
    markers_cycle = markers()
    p0mdata, M_bins, F = p0m(subdf, M, limits=(under_limit,under_limit+5))
    tmp = []
    for g in p0mdata.keys():
        tmp.append(p0mdata[g])
        plt.plot(M_bins, p0mdata[g], label=g, c=color, marker = next(markers_cycle))
    tmp=np.average(tmp, 0)
    try:
        popt, pcov = curve_fit(lambda x, a, b: a - b*x, M_bins, tmp)
        results = results.append(pd.Series(name=under_limit, data=[under_limit, popt[0], popt[1], json.dumps(pcov.tolist()), F[0], F[1]], index=["rank", "a", "b", "cov", "f", "f_"]))
    except:
        print(*sys.exc_info())
del subdf
del M
plt.yscale('log')

plt.xlabel("M", fontsize=24)
plt.ylabel("P(0|M)")

plt.ylim(5e-4,1)

#plt.legend(ncol=3)
fig.savefig("logpm.pdf")
fig.savefig("logpm.png")

In [ ]:
results

In [ ]:
def unpack_covariance_element():
    for result in results["cov"].values:
        pcov = np.array(json.loads(result))
        yield pcov

from matplotlib.ticker import FormatStrFormatter
from scipy.stats import pearsonr

fig, ax = plt.subplots(figsize=(18,15))


ax.scatter(results["f"], results["b"], c="gray", alpha=0.5, ls='--', s = 50**2)

sigma_generator = unpack_covariance_element()
ax.errorbar(results["f"], results["b"], yerr=[np.sqrt(pcov[1,1]) for pcov in sigma_generator], fmt="", lw=0, elinewidth=15)

ax.plot(results["f"], results["f"], ls='--', lw=20)

ax.set_xlim(results["f"].min()*0.8, results["f"].max()*1.5)
ax.set_ylim(results["b"].min()*0.8, results["b"].max()*1.5)

ax.set_xlabel("f", fontsize=35)
ax.set_ylabel("b", fontsize=35)

ax.set_xlim(results["f"][results["f"]>0].min()/10, results["f"].max()*10)
ax.set_ylim(results["b"][results["b"]>0].min()/10, results["b"].max()*10)

ax.set_xscale('log')
ax.set_yscale('log')

ax.xaxis.set_major_formatter(FormatStrFormatter('%.0E'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.0E'))

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", width=5, length=10)

ax.set_title('Correlation: %.3f'%(pearsonr(results["f"],results["b"])[0]), fontsize=35)

plt.tight_layout()
plt.show()
fig.savefig("p0m_fbrelationship.pdf")

In [ ]:
from scipy.optimize import curve_fit

subdf = pd.read_csv("mca/mainTable_Bladder.csv", index_col=0).sample(1000)
print(subdf.info())
M = subdf.sum(0)
M_sorted = M.sort_values(ascending=True)
quantiles = np.quantile(M, q=np.linspace(0,1,10)[:-1])
#quantiles = np.linspace(M.min(), M.max(), 20)
M_classes = pd.Series(index=M.index, data=np.digitize(M, quantiles))

def extrapolate_b(genexpr):
    tmp = np.array([(genexpr[M_classes[M_classes==c].index]==0).astype(int).sum()/len(M_classes[M_classes==c]) for c in np.arange(len(quantiles))+1])
    tmp[np.isnan(tmp)]=0
    popt, pcov = curve_fit(lambda x, a, b: a - b*x, quantiles, tmp)
    del tmp
    return popt[1]
bs = subdf.apply(extrapolate_b, 1)
fs = subdf.mean(1)

In [ ]:
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import pearsonr

fig, ax = plt.subplots(figsize=(18,15))

thr = 2e-1
ax.scatter(fs[fs<thr],bs[fs<thr], c="gray", alpha=0.5, ls='--', s = 200)

#ax.set_xlim(fs[fs>0].min()*0.8, fs[fs<np.inf].max()*1.5)
#ax.set_ylim(bs[bs!=0].min()*0.8, bs[bs<np.inf].max()*1.5)

ax.set_xlabel("f", fontsize=35)
ax.set_ylabel("b", fontsize=35)

ax.xaxis.set_major_formatter(FormatStrFormatter('%.0E'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.0E'))

#ax.set_xscale('log')
#ax.set_yscale('log')


ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", width=5, length=10)

ax.set_title('Correlation: %.3f'%(pearsonr(fs[fs<thr],bs[fs<thr])[0]), fontsize=35)


plt.show()

fig.savefig("p0m_allgenes.pdf")

# Single Tissue

In [ ]:
tissue = "Kidney"
df = pd.read_csv(f"mca/mainTable_{tissue}.csv", index_col=0, header=0)

In [ ]:
M = df.sum(axis=0)
M=M.sort_values(ascending=False)

In [ ]:
M.hist(bins=50)

## Core Size

In [ ]:
quantiles = np.quantile(M[M<4000].values, q=np.linspace(0,1,11))[:-1]
#idxs = np.linspace(500,M.max(),num=20)

sizes = {}
thetas_c = [0.8,0.85,0.9,0.95,0.99]
for theta in thetas_c:
    sizes["%.2f"%theta] = []


#for (m_, m) in zip(M[idxs].index[:-1], M[idxs].index[1:]):
for (m_,m) in zip(quantiles[:-1], quantiles[1:]):
    print(f"[{m_},{m})")
    O = df.reindex(columns=M[(m_<=M)&(M<m)].index).dropna(how="any", axis=1).apply(lambda g: len(g[g>0])/float(len(g)), axis=1)
    for theta in thetas_c:
        sizes["%.2f"%theta].append(len(O[O>theta])/len(O))
    del O

In [ ]:
q_means = (quantiles[1:]+quantiles[:-1])/2.
#q_means = (M[idxs].index[1:]+M[idxs].index[:-1])/2.

M_bin=q_means

N=df.shape[0]

def fit_teo(X_data, gamma, thetac, N=df.dropna(how="all", axis=1).shape[0]):
    """
    - gamma: Zipf exponent
    - theta_c: Occurrences thresholds
    """
    M_bin = X_data
    
    i = np.arange(1,N+1, step=1)
    alpha_i=i**(-gamma)
    alpha=np.sum(alpha_i, axis=0)
    # predicted core size 

    k=M_bin**(1/gamma)/(alpha**(1/gamma)*N)
    c=k*(-np.log(1-thetac))**(-1/gamma)
    return c

In [ ]:
from scipy.optimize import curve_fit
fig,ax = plt.subplots(figsize=(18,15))

for (threshold,sizes_arr), color in zip(sizes.items(),["blue", "gray", "red", "orange", "green"]):
    ax.plot(q_means, sizes_arr, lw=10, marker="o", ms=45, c=color, label=f"data")
    popt = [0.8, float(threshold)]
    ax.plot(q_means, fit_teo(q_means, *popt), c="dark"+color, lw=15, ls="--", label="predicted ($\\theta_c$=%.2f)"%(popt[1]), alpha=0.7)

threshold = 0.9
#ax.plot(q_means, sizes["0.9"], lw=15, marker="o", ms=30, c="gray", label=f"data with thr={threshold}")


#popt, pcov = curve_fit(fit_teo, q_means, sizes["0.9"], p0=[0.8, 0.95])
popt = [0.8, 0.9]
#ax.plot(q_means, fit_teo(q_means, *popt), lw=15, ls="--", label="predicted (gamma=%.2f; thetac=%.2f)"%(popt[0],popt[1]), alpha=0.8, c="orange")

ax.set_ylim(0,0.005)

ax.set_ylabel("Core size", fontsize=45)
ax.set_xlabel("Total reads per cell, $M$", fontsize=45)

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=15)

ax.legend(ncol=2,fontsize=30)
plt.show()
fig.savefig(f"U_core_prediction_{tissue}_allthr.pdf")

In [ ]:
df = df[M.index]
A = df.sum(1)
f = A.sort_values(ascending=False)
#df=df[df.index.isin(f[49:4500].index)]
#save_model(df,tissue="bonemarrow_M100",name="data", n_bins=15)
#mazzolini(np.repeat(M.mean(), len(M)), A/A.sum(), "bonemarrow_M_avg", n_bins=15)

In [ ]:
fig,ax = plt.subplots(figsize=(18,15))
x = np.arange(1, len(f))
ax.set_ylabel("$Frequency, f_i$", fontsize=35)
ax.set_xlabel("$RANK, i$", fontsize=35)
ax.set_yscale('log')
ax.set_xscale('log')
#plt.ylim(1e-7,1)
ax.plot(x, np.power(x,-0.8)*1e-1, 'g--', lw=10, label='$k*i^{-0.8}$')
ax.plot(np.sort(f.dropna())[::-1]/f.sum(), c='blue', lw=15, label='Genes')
if "f_null" in vars().keys():
    ax.plot(np.sort(f_null/f_null.sum())[::-1], c='orange', ls='--', lw=15, label='null_model')
    
ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=15)

ax.legend(fontsize=35)
plt.show()
fig.savefig("zipf.pdf")

## Predict O

In [ ]:
#tissue = "all"
#data = load_all_data()
tissue = "Bone-Marrow_c-kit"
data = load_tissue(tissue)

In [ ]:
P = data["freq"].values
M = data["M"].values
O = data["O"].values
R = M.shape[0]

In [ ]:
O_real = np.array([])
O_pred = np.array([])
O_pred_ds = np.array([])


step = 1500
for start in range(0,len(O),step):
    print(start)
    mask = np.repeat(False, len(O))
    mask[start:start+step]=True

    O_real = np.concatenate([O_real,O[mask]])
    O_pred = np.concatenate([O_pred,list(map(lambda p: 1-1/R*np.sum(np.exp(-p*M)),P[mask]))])
    
genes = data["freq"].index[O_real < (0.8*O_pred)].values

In [ ]:
f = data["freq"]

pos_mask = (~f.isna()) & (f>0)

g_high = f.index[O_pred > O_real + 0.2].values
g_low = f.index[O_pred < O_real * 0.6 - 0.03].values

#mask_high = (f.index.isin(g_high)) & (pos_mask)
#mask_low = (f.index.isin(g_low)) & (pos_mask)
#mask_nc = (f.index.isin(nc)) & (pos_mask)

f_sorted = f[f>0].sort_values(ascending=False)
mask_sorted_high = f_sorted.index.isin(g_high)
mask_sorted_low = f_sorted.index.isin(g_low)

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))

nbins = 25
rang = (0-0.5/nbins, 1+0.5/nbins)

#bins = np.logspace(np.log10(1e-2),np.log10(rang[1]),num=nbins)
bins=np.linspace(1e-4,rang[1],num=nbins)

ax.hist(O, bins=bins, color = "gray", label="data", density=True)

O_pred_hist, bin_edges = np.histogram(O_pred, bins=bins, density=True)
ax.plot((bin_edges[:-1]+bin_edges[1:])/2, O_pred_hist, ls='--', lw=10, c="red", label="O_predicted")
#ax.hist(load_tissue("Bone-Marrow_c-kit",name="mazzolini")["O"], histtype="step", ls='--', bins=bins, density=True, color="orange", lw=10, label="sampling")

ax.plot(bins,[p(x, M.mean(), 0.8, len(O)) for x in bins], lw=15, ls="--", color="darkred", label="teo_prediction <M>")


ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", width=5, length=10)

ax.set_yscale('log')
#ax.set_xscale('log')

ax.set_ylim(5e-4,1e2)

ax.legend(fontsize=35)
ax.set_xlabel("Occurrence, $O_i$", fontsize=35)
ax.set_ylabel("pdf", fontsize=35)

plt.tight_layout()
plt.show()
fig.savefig(f"U_{tissue}_sampling_pred.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))
ax.scatter(O_real, O_pred, s=200, c='gray')
ax.plot([0,1], [0,1], color = "black", ls='--', lw=15, alpha=0.9)

ax.scatter(O_real[mask_high], O_pred[mask_high], s=200, c='b')
#ax.scatter(O_real[mask_low], O_pred[mask_low], s=200, c='red')

#ax.scatter(O_real[mask_nc], O_pred[mask_nc], s=200, c='red')
#ax.scatter(O_pred, load_tissue(tissue, "mazzolini")["O"], s=200, c='red')


#ax.set_xlabel("$O_i$ from data", fontsize=35)
ax.set_xlabel("$O_i$ from from sampling", fontsize=35)
ax.set_ylabel("$O_i$ predicted", fontsize=35)

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", width=5, length=10)

plt.tight_layout()
plt.show()
fig.savefig(f"O_real-O_pred_{tissue}.pdf")

In [ ]:
with open(f"OrealOpred_overlist_{tissue}_low.txt","w") as f:
    list(map(lambda g: f.write(g+"\n"), filter(lambda g: "Rik" not in g, data["freq"].index.values[mask_low])))

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))

f_sorted.plot(ax=ax, color="gray", lw=15, label="data")

for mask, label, color, marker in zip([mask_sorted_low, mask_sorted_high],
                                      ["genes under $O_i$ prediction", "genes over $O_i$ prediction"],
                                     ["r", "b"],
                                     ["o","o"]):
    count = mask.sum()
    ax.plot(np.arange(0,len(data["freq"]),1)[mask], f_sorted[mask].sort_values(ascending=False), lw=0, ms=20, label=label + f"({count})", c=color, marker=marker, alpha = 0.4)

ax.set_xscale("log")
ax.set_yscale("log")


ax.set_ylabel("$Frequency, f_i$", fontsize=35)
ax.set_xlabel("$RANK, i$", fontsize=35)
ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=15)

ax.legend(fontsize=35)

In [ ]:
df = pd.read_csv(f"mca/mainTable_{tissue}.csv", index_col=0)

In [ ]:
#c_mat=df[df.index.isin(g_high)].transpose().applymap(lambda x: 1 if x > 0 else 0).corr()
c_mat=df[df.index.isin(g_high)].transpose().applymap(lambda x: 1 if x > 0 else 0).corr(method = lambda x,y: np.abs(x-y).sum()/len(x))

In [ ]:
sns.heatmap(c_mat, vmin=0, vmax=1, xticklabels=g_high, yticklabels=g_high)

In [ ]:
import tensorflow as tf
import time

In [ ]:
subdf = df.sample(len(g_high), axis=0).append(df[df.index.isin(g_high)])

In [ ]:
data = tf.convert_to_tensor(subdf.applymap(lambda x: 1 if x > 0 else 0).values)
start = time.time()
corr_mat = tf.map_fn(lambda x: tf.map_fn(lambda y: tf.divide(tf.reduce_sum(tf.abs(x-y)), x.shape[0], tf.float32), data, dtype=tf.float64), data, dtype=tf.float64)
print(time.time()-start)

In [ ]:
start = time.time()
c_mat = subdf.transpose().applymap(lambda x: 1 if x > 0 else 0).corr(method = lambda x,y: np.abs(x-y).sum()/len(x))
print(time.time()-start)

In [ ]:
#subdf = df.sample(len(g_high), axis=0)
#subdf = df[df.index.isin(g_high)]
#c_mat = subdf.transpose().applymap(lambda x: 1 if x > 0 else 0).corr(method = lambda x,y: np.abs(x-y).sum()/len(x))
#c_mat = subdf.transpose().applymap(lambda x: 1 if x > 0 else 0).corr()
sns.heatmap(c_mat, vmin=0, vmax=1, xticklabels=subdf.index, yticklabels=subdf.index)

In [ ]:
fig,ax = plt.subplots()
x=data["means"]
ax.scatter(x,data["cv2"])
ax.scatter(x,data["mazzolini"]["cv2"])
ax.plot(x,1/x)

ax.set_xscale("log")
ax.set_yscale("log")

## Heaps

In [ ]:
def h(m, gamma, N):
    i = np.arange(1, N+1, step=1)
    alpha_i = i**(-gamma)
    alpha = np.sum(alpha_i, axis=0)
    return N - np.sum((1-np.power(i, -gamma)/alpha)**m, axis=0)

def h_double(m, gamma: list, N, F=122):
    g1, g2 = gamma
    i = np.arange(1, N+1, step=1)
    alpha_i = [_i**(-g1) if _i < F else _i**(-g2) for _i in i]
    alpha = np.sum(alpha_i, axis=0)
    return N - np.sum((1-np.power(i[:F], -g1)/alpha)**m, axis=0) - np.sum((1-np.power(i[F:], -g2)/alpha)**m, axis=0)

def h_double_cut(m, gamma: list, N, F=122, F2=4100):
    g1, g2 = gamma
    i = np.arange(1, F2+1, step=1)
    alpha_i = [_i**(-g1) if _i < F else _i**(-g2) for _i in i]
    alpha = np.sum(alpha_i, axis=0)
    return F2 - np.sum((1-np.power(i[:F], -g1)/alpha)**m, axis=0) - np.sum((1-np.power(i[F:], -g2)/alpha)**m, axis=0)

def h_triple(m, gamma: list, N, F1=122, F2=4100):
    g1, g2 = gamma
    i = np.arange(1, N+1, step=1)
    
    b = 1./(F1**(g1-g2)*np.sum([_i**(-g1) for _i in i[:F1]])+np.sum([_i**(-g2) for _i in i[F1:F2]]) + (F2**(-g2))*np.exp(0.002*F2)*np.sum([np.exp(-0.002*_i) for _i in i[F2:]]))
    c = b * (F2**(-g2))*np.exp(0.002*F2)
    a = b * F1**(g1-g2)
    return N - np.sum((1-a*np.power(i[:F1], -g1))**m, axis=0) - np.sum((1-b*np.power(i[F1:F2], -g2))**m, axis=0) - np.sum((1-c*np.exp(-0.002*i[F2:]))**m, axis=0)

def h_rho(m,f: np.array,N):
    return N - np.sum(np.exp(-f/f.sum()*m))

In [ ]:
fig, ax = plt.subplots(figsize=(28,24))

skip_bins=5
ax.scatter(data["M"], data["diffWords"], color="gray", s=200, label="data")

#ax.scatter(data["nullteo"]["M"], data["nullteo"]["diffWords"], color="orange", s=250, label="sampling_i**-0.8")
ax.scatter(data["mazzolini"]["M"], data["mazzolini"]["diffWords"], color="darkorange", s=250, label="sampling_ensamble")


bin_means, bin_edges, binnumber = stats.binned_statistic(data["M"], data["diffWords"],statistic='mean', bins=np.logspace(np.log10(data["M"].min()),np.log10(max(data["M"])), 25))

ax.plot((bin_edges[:-1][:-skip_bins] + bin_edges[1:][:-skip_bins])/2, bin_means[:-skip_bins], color='blue', lw=10, marker="o", ms=25, label='binned average')


x_bins=np.logspace(np.log10(data["M"].min()),np.log10(max(data["M"])))
ax.plot(x_bins, [h(x, 0.86, len(f.dropna())) for x in x_bins], lw=15, alpha=0.6, ls="--", c="orange", label="conto mazzolini")
ax.plot(x_bins, [h_double(x, [0.5, 0.9], len(data["freq"].dropna()),F=122) for x in x_bins], lw=15, alpha=0.6, ls="--", c="darkred", label="mazzolini double_scaling")
ax.plot(x_bins, [h_double_cut(x, [0.5, 0.9], len(data["freq"].dropna()),F=122) for x in x_bins], lw=15, alpha=0.6, ls="--", c="darkorange", label="mazzolini double_scaling_cut")
ax.plot(x_bins, [h_triple(x, [0.5, 1.0], len(data["freq"].dropna()), F1=122, F2=4100) for x in x_bins], lw=15, alpha=0.6, ls="--", c="red", label="mazzolini triple_scaling")
#ax.plot(x_bins, [h_rho(x, f.dropna(), len(f.dropna())) for x in x_bins], lw=15, alpha=0.6, ls="--", c="purple", label="$N-\Sigma_i P_i(0|M_s)$")


ax.set_ylabel("Number of genes expressed", fontsize=35)
ax.set_xlabel("UMI per cell, $M_s$", fontsize=35)
ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=15)

ax.legend(fontsize=25)

ax.set_xscale("log")
ax.set_yscale("log")

plt.show()
fig.savefig(f"heaps_{tissue}_mazzolini_teo.pdf")

In [ ]:
X = np.linspace(1,3e4, 30000)
f=1/X
f=f/sum(f)
plt.plot(X,f, lw=10)

f=[x**-0.5 if x < 122 else x**(-1) for x in X]
f=f/sum(f)
plt.plot(X,f, lw=8, alpha = 0.4)

f=[x**-0.5 if x < 122 else x**(-1) for x in X[:4100]]
f=f/sum(f)
plt.plot(X[:4100],f, lw=6, ls=":", alpha = 0.4)

f=[x**-0.5 if x < 122 else x**(-1) if x < 4100 else np.exp(-0.002*x) for x in X]
f=f/sum(f)
plt.plot(X,f, lw=4, ls="--", alpha = 0.4)

plt.xscale("log")
plt.yscale("log")
plt.ylim(1e-10,1)
plt.show()

# Load Null model

In [ ]:
tissue = "Bone-Marrow_c-Kit"
#tissue = "bonemarrow_M100"

data = load_tissue(tissue, "data")

for method in ["nullteo","mazzolini"]:
    try:
        data[method] = load_tissue(tissue, method)

        means_null = data[method]['means']
        var_null = data[method]['var']
        f_null = data[method]['freq']
        f_null[np.isnan(f_null)]=0
        O_null = data[method]['O']
        M_null = data[method]["M"]
        cv2_null = data[method]['cv2']
        diffWords_null = data[method]['diffWords']
        means_nozero_null = data[method]['means_nonzero']
    except:
        print(*sys.exc_info())
    

means = data['means']
var = data['var']
f = data['freq']
O = data['O']
M = data["M"]
cv2 = data['cv2']
diffWords = data['diffWords']
means_nozero = data['means_nonzero']

#mazzolini(M, f, tissue)

In [ ]:
from frontiers_analysis import null_model
null_model(M.sample(500), f, tissue)
os.system(f"mv data_{tissue}_null.pkl mca/data_{tissue}_null.pkl")


null_model(M.sample(500), pd.Series(index=["g%d"%g for g in range(len(f))], data = [(i+1)**-0.8 for i in range(len(f))]), tissue)
os.system(f"mv data_{tissue}_null.pkl mca/data_{tissue}_nullteo.pkl")


#mazzolini(M.sample(500),f,tissue)

In [ ]:
fig = plt.figure(figsize=(15,8))
x = np.arange(1, len(f))
plt.ylabel("$Frequency, f_i$", fontsize=24)
plt.xlabel("$RANK, i$", fontsize=24)
plt.yscale('log')
plt.xscale('log')
#plt.ylim(1e-7,1)
plt.plot(x, 1./x, 'g--', lw=2, label='$i^{-1}$')
plt.plot(np.sort(f)[::-1], c='blue', lw=8, label='Genes')
plt.plot(np.sort(f_null)[::-1], c='orange', ls='--', lw=6, label='null_model')
plt.legend(fontsize=24)
plt.show()
fig.savefig("zipf_null_0.pdf")
fig.savefig("zipf_null_0.png")

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.scatter(M, diffWords, label='samples')
plt.scatter(M_null, diffWords_null, label='null_model')
plt.xlabel("Realization size", fontsize=22)
plt.ylabel("#different words", fontsize=22)
#bin_means, bin_edges, binnumber = stats.binned_statistic(M, diffWords,statistic='mean', bins=np.linspace(0,max(M)))
#plt.hlines(bin_means[:-15], bin_edges[:-1][:-15], bin_edges[1:][:-15], colors='r', lw=5, label='binned average')
#plt.xscale('log')
#plt.yscale('log')
plt.xlim(0,max(M)*1.1)
plt.legend(fontsize=20)
plt.show()
fig.savefig("heaps_null_o.pdf")
fig.savefig("heaps_null_o.png")

In [ ]:
fig, ax =plt.subplots(figsize=(15,8))
ax.scatter(means_null.reindex_like(means.drop_duplicates()), cv2_null.reindex_like(means.drop_duplicates()), c='orange', label='mazzolini')
plotcv2mean(means.values, var.values, ax=ax, normalisation_str='counts')
fig.savefig("cvmean_loglog.pdf")

In [ ]:
bins = 10
rang = (0-0.5/bins, 1+0.5/bins)
fig, ax =plt.subplots(1,2, figsize=(20,10))
ax[0].hist(np.array(O.dropna(), dtype=float), histtype='step', lw=5, range = rang, color='blue', density=True, bins=bins, label='data')
if "O_null" in vars().keys():
    ax[0].hist(np.array(O_null, dtype=float), histtype='step', lw=5, range = rang, bins=bins, color='orange', ls='--', density=True, label='Mazzolini')
    
if "O_null_1" in vars().keys():
    ax[0].hist(np.array(O_null_1, dtype=float), histtype='step', lw=5, range = rang, bins=bins, color='green', ls='--', density=True, label='null_1')
ax[0].set_xlabel("$o_i$", fontsize=24)
ax[0].set_ylabel("pdf", fontsize=24)
ax[0].legend(fontsize=20)

ax[1].hist(np.array(O.dropna(), dtype=float), histtype='step', lw=5, range = rang, color='blue', density=True, bins=bins, label='data')
if "O_null" in vars().keys():
    ax[1].hist(np.array(O_null, dtype=float), histtype='step', lw=5, range = rang, bins=bins, color='orange', ls='--', density=True, label='Mazzolini')
    
if "O_null_1" in vars().keys():
    ax[1].hist(np.array(O_null_1, dtype=float), histtype='step', lw=5, range = rang, bins=bins, color='green', ls='--', density=True, label='null_1')
ax[1].set_xlabel("$o_i$", fontsize=24)
ax[1].set_ylabel("pdf", fontsize=24)
ax[1].legend(fontsize=20)

ax[1].set_yscale('log')
ax[1].set_xscale('log')

plt.show()
fig.savefig("U_null_0.pdf")
fig.savefig("U_null_0.png")